In [ ]:
%pip installpandas matplotlib

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from pathlib import Path

In [ ]:
PROJECT_DIR = Path().resolve().parent.parent
DATA_DIR = PROJECT_DIR / "data" / "raw"
REPORTS_DIR = PROJECT_DIR / "eda" / "reports"
participants = ["LL", "ZM", "VV", "YL", "YT"] 
activities = ["sitting", "walking", "running", "lying"]
sensors = ["AX", "AY", "AZ", "PI", "PG", "PR", "HR"]
RESAMPLE_FREQ = "40ms"

In [ ]:
def load_sensor_file(participant, sensor, activity):
    
    matching_files = [f for f in DATA_DIR.iterdir() if f.is_file()
                      and participant.lower() in f.name.lower()
                      and sensor.lower() in f.name.lower()
                      and activity.lower() in f.name.lower()]
    
    if not matching_files:
        print(f"Warning: No file found for {participant}-{sensor}-{activity}")
        return None
    

    file = matching_files[0]
    df = pd.read_csv(file)
    
    ts_unit = 's'
    df = df[['LocalTimestamp', sensor]].copy()
    df['LocalTimestamp'] = pd.to_datetime(df['LocalTimestamp'], unit=ts_unit, utc=True)
    df['LocalTimestamp'] = df['LocalTimestamp'].dt.tz_convert('America/Montreal')
    df = df.set_index('LocalTimestamp')
    return df

def resample_sensor(df, freq=RESAMPLE_FREQ):
    numeric_df = df.select_dtypes(include='number')
    return numeric_df.resample(freq).mean().interpolate()

def plot_sensor(df, participant, activity, sensor):
    if df is None or df.empty:
        return
    plt.figure(figsize=(10, 4))
    plt.plot(df.index, df[df.columns[0]])
    plt.title(f"{participant.upper()} - {activity.capitalize()} - {sensor.upper()}")
    plt.xlabel("Time")
    plt.ylabel("Signal")
    filename = REPORTS_DIR / f"{participant}_{activity}_{sensor}.png"
    plt.savefig(filename)
    plt.close()


In [10]:
for participant in participants:
    for activity in activities:
        sensor_dfs = {}
        for sensor in sensors:
            try:
                df = load_sensor_file(participant, sensor, activity)
                if df is not None and not df.empty:
                    print(f"Loaded {participant}-{sensor}-{activity}, shape: {df.shape}")
                    df_resampled = resample_sensor(df)
                    sensor_dfs[sensor] = df_resampled
                    plot_sensor(df_resampled, participant, activity, sensor)

            except FileNotFoundError:
                continue

Loaded LL-AX-sitting, shape: (3253, 1)
Loaded LL-AY-sitting, shape: (3253, 1)
Loaded LL-AZ-sitting, shape: (3253, 1)
Loaded LL-PI-sitting, shape: (3236, 1)
Loaded LL-PG-sitting, shape: (3236, 1)
Loaded LL-PR-sitting, shape: (3236, 1)
Loaded LL-HR-sitting, shape: (98, 1)
Loaded LL-AX-walking, shape: (3154, 1)
Loaded LL-AY-walking, shape: (3154, 1)
Loaded LL-AZ-walking, shape: (3154, 1)
Loaded LL-PI-walking, shape: (3138, 1)
Loaded LL-PG-walking, shape: (3138, 1)
Loaded LL-PR-walking, shape: (3138, 1)
Loaded LL-HR-walking, shape: (197, 1)
Loaded LL-AX-running, shape: (3227, 1)
Loaded LL-AY-running, shape: (3227, 1)
Loaded LL-AZ-running, shape: (3227, 1)
Loaded LL-PI-running, shape: (3210, 1)
Loaded LL-PG-running, shape: (3210, 1)
Loaded LL-PR-running, shape: (3210, 1)
Loaded LL-HR-running, shape: (297, 1)
Loaded LL-AX-lying, shape: (3126, 1)
Loaded LL-AY-lying, shape: (3126, 1)
Loaded LL-AZ-lying, shape: (3126, 1)
Loaded LL-PI-lying, shape: (3109, 1)
Loaded LL-PG-lying, shape: (3109, 1)


In [ ]:
sensor = "HR"

for participant in participants:
    print(f"\n=== Participant {participant.upper()} ===")
    for activity in activities:
        df = load_sensor_file(participant, sensor, activity)
        if df is not None and not df.empty:
            df_resampled = resample_sensor(df)
            hr_series = df_resampled[sensor]
            
            hr_max = hr_series.max()
            hr_min = hr_series.min()
            hr_mean = hr_series.mean()
            
            print(f"{activity.capitalize()}: Max HR = {hr_max:.1f}, "
                  f"Min HR = {hr_min:.1f}, Avg HR = {hr_mean:.1f}")
        else:
            print(f"{activity.capitalize()}: No data available")